In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 10
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error  
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

***
부스팅(Boosting)
***

초기에는 모든 데이터 포인트에 동일한 가중치를 할당하고 점차 학습이 진행되면서 올바르게 분류된 데이터 포인트의 가중치는 감소시키는 반면,  
잘못 분류된 데이터 포인트의 가중치는 증가시킨다.  
결과적으로 학습이 진행되면서 학습기는 잘못 분류된(분류하기 어려운) 데이터에 집중하게 되고,  
이전 단계에서 만들어진 학습기는 다음 단계에서 사용할 학습 데이터의 가중치를 반영하는데 사용하므로 부스팅은 배깅과는 달리 이전 분류기의 영향을 받는다.  

배깅은 각 데이터 포인트가 추출될 확률이 모두 동일하지만, 부스팅은 각 데이터 포인트에 할당된 가중치에 비례해서 추출된다.  

***
에이다 부스트(Aida Boost)
***
약한 학습기 여러개를 모아 하나의 강한 학습기를 만드는 방법으로 개별적으로는 약한 학습 모델이지만, 이와 같은 모델을 다수 생성하고 부스팅을 적용함으로써  
강한 학습기가 만들어진다.  

보팅이낭 배깅은 모델이 병렬적으로 실행된다. 10개의 모델이 있다면 10개의 모델을 동시에 학습시킬 수 있다는 뜻이다.  
이에 반해, 부스팅은 여러 약한 학습기가 순차적으로 적용된다. 그 이유는 약한 학습 이후 판별하지 못한 데이터 포인트에 대해서 가중치를 부여하기 때문이다.   

에이다 부스트의 핵심 아이디어는 분류하기 어려운 학습 데이터에 가중치를 더 높이는 것이다. 즉, 이전에 잘못 분류된 학습 데이터 포인트는 가중치가 증가해 오차율이 높아진다.  
다음 약한 학습기는 이전 학습기에서 증가한 오차를 낮추는 방향으로 학습하게 된다.   

에이다 부스트는 일반적인 부스팅과는 다르게 약한 학습기를 학습할 때 학습 데이터셋 전체를 사용한다.  
학습 샘플은 반복할 때마다 가중치가 부여되며 이 앙상블은 이전 학습기가 실수한 부분을 학습하는 강력한 분류기를 만든다.

***
암 데이터를 사용해 암 여부를 예측하는 모델을 생성하고 학습한다.
***

In [2]:
# 데이터 불러오기
raw_data = datasets.load_breast_cancer() # 사이킷런이 제공하는 암 데이터를 불러온다.

# 피쳐, 레이블 데이터 저장
xData = raw_data.data # 피쳐 데이터를 저장한다.
yData = raw_data.target # 피쳐 데이터에 따른 레이블을 저장한다.
print(xData.shape, yData.shape)


# 학습 데이터와 테스트 데이터로 분할
x_train, x_test, y_train, y_test = train_test_split(xData, yData, random_state=0)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

# 데이터 표준화
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
x_train = std_scale.fit_transform(x_train) # 학습 데이터를 스케일러로 표준화하고 적용한다.
x_test = std_scale.transform(x_test) # 테스트 데이터를 학습 데이터로 표준화된 스케일러에 적용한다.

# 모델 생성 후 데이터 학습
from sklearn.ensemble import AdaBoostClassifier # 에이다 부스트 알고리즘 사용

# 모델을 만든다.
clf = AdaBoostClassifier()

# 표준화된 학습 데이터 x_train와 학습 데이터에 따른 레이블 데이터 y_train으로 앙상블 에이다 부스트 모델을 학습시킨다.
clf.fit(x_train, y_train)


(569, 30) (569,)
(426, 30) (143, 30) (426,) (143,)


AdaBoostClassifier()

***
학습된 모델로 테스트 데이터를 예측한다.
***

In [3]:
#predict() 메소드의 인수로 표준화된 테스트 데이터를 넘겨서 예측한다.
predict = clf.predict(x_test)
print(predict)

[0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0
 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 0 0 1
 0 0 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 0]


***
학습된 모델을 평가한다
***

In [4]:
# 혼동 행렬
conf_matrix = confusion_matrix(y_test, predict)
print(conf_matrix)

[[52  1]
 [ 2 88]]


In [5]:
#분류 리포트
class_report = classification_report(y_test, predict, target_names=raw_data.target_names)
print(class_report)

              precision    recall  f1-score   support

   malignant       0.96      0.98      0.97        53
      benign       0.99      0.98      0.98        90

    accuracy                           0.98       143
   macro avg       0.98      0.98      0.98       143
weighted avg       0.98      0.98      0.98       143

